## EV Manufacturer Suggested Retail Prices (by Models)

**Data Source: US Department of Energy - Office of Energy efficiency & Renewable Energy**

URL: https://www.fueleconomy.gov/feg/PowerSearch.do?action=PowerSearch&year1=2016&year2=2023&minmsrpsel=0&maxmsrpsel=0&city=0&highway=0&combined=0&cbvtelectric=Electric&YearSel=2016-2023&MakeSel=&MarClassSel=&FuelTypeSel=&VehTypeSel=Electric&TranySel=&DriveTypeSel=&CylindersSel=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=50

Data Description: 

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

In [11]:
# List of URLs for multiple pages
urls = [
    'https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&year1=2016&year2=2023&minmsrpsel=0&maxmsrpsel=0&city=0&hwy=0&comb=0&cbvtelectric=Electric&YearSel=2016-2023&make=&mclass=&vfuel=&vtype=Electric&trany=&drive=&cyl=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=200&pageno=1&tabView=2',
    'https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&year1=2016&year2=2023&minmsrpsel=0&maxmsrpsel=0&city=0&hwy=0&comb=0&cbvtelectric=Electric&YearSel=2016-2023&make=&mclass=&vfuel=&vtype=Electric&trany=&drive=&cyl=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=200&pageno=2&tabView=2',
    'https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&year1=2016&year2=2023&minmsrpsel=0&maxmsrpsel=0&city=0&hwy=0&comb=0&cbvtelectric=Electric&YearSel=2016-2023&make=&mclass=&vfuel=&vtype=Electric&trany=&drive=&cyl=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=200&pageno=3&tabView=2'
]

# Loop over each URL to fetch data from multiple pages
for url in urls:
    # Send request and parse HTML
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all 'tr' tags with the 'ymm-row' class
    vehicle_rows = soup.find_all('tr', class_='ymm-row')

    # Loop over each 'ymm-row' and find the next row for cost and msrp
    for vehicle_row in vehicle_rows:
        # Get vehicle details
        vehicle_tag = vehicle_row.find('a')
        if vehicle_tag:
            vehicle_text = vehicle_tag.get_text(strip=True)
            year, model = vehicle_text.split(' ', 1)  # Split on the first space to separate year and model
            years.append(year)
            models.append(model)
        else:
            years.append(np.nan)
            models.append(np.nan)

        # Find the next sibling row for cost and MSRP
        cost_row = vehicle_row.find_next_sibling('tr')
        if cost_row:
            fuel_cost_tag = cost_row.find('td', class_='ann-fuel-cost')
            msrp_tag = cost_row.find('td', class_='msrp')

            # Check if MSRP exists, else skip this entry
            if msrp_tag:
                # Process fuel cost
                if fuel_cost_tag:
                    fuel_cost_text = fuel_cost_tag.get_text(strip=True)
                    annual_fuel_costs.append(fuel_cost_text.replace('\r', '').replace('\n', '').replace('\t', ''))
                else:
                    annual_fuel_costs.append(np.nan)

                # Process MSRP and clean up
                msrp_text = msrp_tag.get_text(strip=True)
                msrp_cleaned = msrp_text.replace('\r', '').replace('\n', '').replace('\t', '')
                msrp_prices.append(msrp_cleaned)
            else:
                # If MSRP is missing, skip this entry
                years.pop()  # Remove the last added item from years
                models.pop()  # Remove the last added item from models
        else:
            # If there's no next row, remove the last entry
            years.pop()
            models.pop()

# Create DataFrame
data = {
    'Year': years,
    'Model': models,
    'Annual Fuel Cost': annual_fuel_costs,
    'MSRP': msrp_prices
}
df = pd.DataFrame(data)

# Convert 'Year' to numeric for sorting, ignoring errors for NaN
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

# Sort by Year
df_sorted = df.sort_values(by='Year', ascending=True)

# Display the DataFrame
print(df_sorted)

     Year                                   Model Annual Fuel Cost  \
357  2016       smart fortwo electric drive coupe             $700   
94   2016     Nissan Leaf (24 kW-hr battery pack)             $650   
385  2016                 Tesla Model S AWD - 60D             $700   
369  2016                       Kia Soul Electric             $700   
29   2016                              BMW i3 BEV             $600   
..    ...                                     ...              ...   
130  2023           Tesla Model Y Performance AWD             $700   
458  2023  BMW i4 M50 Gran Coupe (19 inch wheels)             $800   
460  2023                    Genesis GV60 ADVANCE             $800   
431  2023     Ford MUSTANG MACH-E CAL RT 1 ER AWD             $750   
641  2023                     Lordstown Endurance           $1,600   

                 MSRP  
357                    
94   $29,010– $36,790  
385           $71,000  
369  $31,950– $35,950  
29            $42,400  
..             

In [20]:
# Get the current working directory instead of using __file__
current_dir = os.getcwd()
csv_file_path = os.path.join(current_dir, 'csv_data_files', 'ev_pricing_data.csv')

df_sorted.to_csv(csv_file_path, index = False)

# Read the CSV file into a DataFrame
pricing_csv = pd.read_csv(csv_file_path)
print(pricing_csv)

     Year                                   Model Annual Fuel Cost  \
0    2016       smart fortwo electric drive coupe             $700   
1    2016     Nissan Leaf (24 kW-hr battery pack)             $650   
2    2016                 Tesla Model S AWD - 60D             $700   
3    2016                       Kia Soul Electric             $700   
4    2016                              BMW i3 BEV             $600   
..    ...                                     ...              ...   
637  2023           Tesla Model Y Performance AWD             $700   
638  2023  BMW i4 M50 Gran Coupe (19 inch wheels)             $800   
639  2023                    Genesis GV60 ADVANCE             $800   
640  2023     Ford MUSTANG MACH-E CAL RT 1 ER AWD             $750   
641  2023                     Lordstown Endurance           $1,600   

                 MSRP  
0                 NaN  
1    $29,010– $36,790  
2             $71,000  
3    $31,950– $35,950  
4             $42,400  
..             